# Agentic AI Prototype — ERP + CMM Integration (Local Mode)

This notebook runs the local-agent pipeline: schema mapping, integration, and analysis.
It uses the merged dataset `outputs/unified_dataset_sample.csv`.


In [ ]:
import pandas as pd
from pathlib import Path
DATA_DIR = Path('..') / 'outputs'
merged = pd.read_csv(DATA_DIR / 'unified_dataset_sample.csv', low_memory=False)
merged.head()

## Quick summary statistics

In [ ]:
print('Rows:', len(merged))
print('Columns:', merged.columns.tolist())
if 'pass_flag' in merged.columns:
    print('\nPass flag distribution:')
    print(merged['pass_flag'].value_counts(dropna=False))
else:
    print('No pass_flag column present')

## Visualizations

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

# Pass/Fail Pie
pf = merged['pass_flag'].map({True:'pass', False:'fail'}).fillna('unknown')
pf_counts = pf.value_counts()
plt.figure(figsize=(4,4))
pf_counts.plot.pie(autopct='%1.1f%%', ylabel='')
plt.title('Pass / Fail Distribution')
plt.show()

In [ ]:
# --- Fail Rate by Feature ---
if "result" in merged.columns and "feature_name" in merged.columns:
    fail_by_feature = (
        merged[merged["result"].str.lower() == "fail"]
        .groupby("feature_name")
        .size()
        .div(merged.groupby("feature_name").size())
        .fillna(0)
        .sort_values(ascending=False)
    )

    print("Fail rate by feature:")
    display(fail_by_feature)

    plt.figure(figsize=(8,5))
    fail_by_feature.plot(kind="barh", color=plt.cm.tab10(0))
    plt.xlabel("Fail Rate")
    plt.ylabel("Feature")
    plt.title("Fail Rate by Feature")
    plt.tight_layout()
    plt.show()
else:
    print(" 'result' or 'feature_name' column not found in dataset.")

### Top 10 Defective Lots by Defect Rate

In [ ]:
# Create lot_summary: defects and total_parts per lot
lot_summary = (
    merged.groupby('lot_id_cmm')
    .agg(defects=('pass_flag', lambda x: (~x).sum()), total_parts=('pass_flag', 'size'))
)

# Top 10 lots by absolute defects
top_defects = lot_summary.sort_values('defects', ascending=False).head(10)

plt.figure(figsize=(10,6))
plt.barh(top_defects.index, top_defects['defects'], color="gray")

# Add defect counts as labels
for i, (defects, total) in enumerate(zip(top_defects['defects'], top_defects['total_parts'])):
    plt.text(defects + 0.2, i, f"{defects}/{total} fails", va='center')

plt.xlabel('Number of Defects')
plt.ylabel('Lot ID')
plt.title('Top 10 Defective Lots by Defect Count')
plt.tight_layout()
plt.show() #since no more than 1 value and 1 defect is present in these lots, therefore it is showing like all the bars are of same length - 1/1 fails.

### Anomaly Scatter (if present)

In [ ]:
# Define the feature and value column to analyze
feature_to_analyze = "outer_diameter"  
val_col = "measured_value"

# Filter merged dataframe for the selected feature
df_an = merged[merged["feature_name"] == feature_to_analyze].copy()

df_an['deviation'] = df_an[val_col] - df_an[val_col].mean()

plt.figure(figsize=(10,4))
plt.scatter(df_an.index, df_an['deviation'], c="blue", s=5, alpha=0.5, label="Normal")
plt.scatter(df_an[df_an['anomaly_iforest']==True].index,
            df_an[df_an['anomaly_iforest']==True]['deviation'],
            c="red", s=10, label="Anomaly")
plt.axhline(0, color="black", linestyle="--")
plt.title(f"Anomalies by Deviation in {val_col}")
plt.xlabel("Sample Index")
plt.ylabel("Deviation from Mean")
plt.legend()
plt.tight_layout()
plt.show()